In [1]:
from erddapy import ERDDAP # /anaconda3/envs/colocators-ohw19/lib/python3.7/site-packages/erddapy
from erddapy import utilities

In [2]:
import json
import urllib
import requests
file = urllib.request.urlopen('https://raw.githubusercontent.com/IrishMarineInstitute/search-erddaps/master/erddaps.json')
all_servers = json.loads(file.read())
#file.getcode()

In [3]:
servers=all_servers

In [4]:
del servers[16] # skipping scoos server
#del servers[4]
len(servers)

27

In [5]:
#from lxml import etree
#cf_standard_names = urllib.request.urlopen('http://cfconventions.org/Data/cf-standard-names/67/src/cf-standard-name-table.xml')
#xml = etree.XML(cf_standard_names.read())
#print(xml.get("id"))
#std_names=[]
#for child in xml:
#    if child.get("id") is not None:
#        std_names.extend(child.get("id"))
#get all entry id="
#print(len(std_names))
import ipywidgets as widgets

std_names=['sea_water_salinity','sea_water_temperature','mass_concentration_of_chlorophyll_in_sea_water','eastward_sea_water_velocity','northward_sea_water_velocity']


drpdwn = widgets.Dropdown(
    options=std_names,
    description='CF Standard Names:',
    disabled=False,
)
drpdwn

Dropdown(description='CF Standard Names:', options=('sea_water_salinity', 'sea_water_temperature', 'mass_conce…

In [6]:

time_min = '2016-07-10T00:00:00Z'
time_max = '2016-08-10T00:00:00Z'
bbox = [-72.0, -69, 38, 41]

kw = {
   'search_for': 'all',
   'min_lon': bbox[0],
   'max_lon': bbox[1],
   'min_lat': bbox[2],
   'max_lat': bbox[3],
   'min_time': time_min,
   'max_time': time_max,
   'standard_name': drpdwn.value
}

In [7]:
import pandas as pd
all_datasets=pd.DataFrame()

for key in servers:
    url = key['url']
    url = url.rstrip("/")
    e = ERDDAP(
                 server=url,
                 protocol='tabledap',
                 response='csv'
           )
    r = requests.get(e.get_search_url(**kw))
    try:
        r.raise_for_status()
        print(e.get_search_url(**kw))
        ds = pd.read_csv('%s'%e.get_search_url(**kw))
        ds['server'] = url
    
        datasets = ds[['server','Dataset ID','tabledap','Institution','Summary']]
    
        datasets.dropna(subset=['tabledap'],inplace=True)
    
        all_datasets = pd.concat([all_datasets,datasets])
    except requests.exceptions.HTTPError as err:
        print("Bad ERDDAP!!! {}".format(url))
    except requests.exceptions.RequestException as err:
        print("Bad ERDDAP!!! {}".format(url))
    except requests.exceptions.ConnectionError as err:
        print("Bad ERDDAP!!! {}".format(url))
    except requests.exceptions.Timeout as err:
        print("Bad ERDDAP!!! {}".format(url))
    except requests.exceptions.ConnectTimeout as err:
        print("Bad ERDDAP!!! {}").format(url)
        


https://coastwatch.pfeg.noaa.gov/erddap/search/advanced.csv?page=1&itemsPerPage=1000&protocol=(ANY)&cdm_data_type=(ANY)&institution=(ANY)&ioos_category=(ANY)&keywords=(ANY)&long_name=(ANY)&standard_name=sea_water_salinity&variableName=(ANY)&minLon=-72.0&maxLon=-69&minLat=38&maxLat=41&minTime=1468108800.0&maxTime=1470787200.0&searchFor=all


/anaconda3/envs/colocators-ohw19/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


https://apdrc.soest.hawaii.edu/erddap/search/advanced.csv?page=1&itemsPerPage=1000&protocol=(ANY)&cdm_data_type=(ANY)&institution=(ANY)&ioos_category=(ANY)&keywords=(ANY)&long_name=(ANY)&standard_name=sea_water_salinity&variableName=(ANY)&minLon=-72.0&maxLon=-69&minLat=38&maxLat=41&minTime=1468108800.0&maxTime=1470787200.0&searchFor=all
https://erddap.bco-dmo.org/erddap/search/advanced.csv?page=1&itemsPerPage=1000&protocol=(ANY)&cdm_data_type=(ANY)&institution=(ANY)&ioos_category=(ANY)&keywords=(ANY)&long_name=(ANY)&standard_name=sea_water_salinity&variableName=(ANY)&minLon=-72.0&maxLon=-69&minLat=38&maxLat=41&minTime=1468108800.0&maxTime=1470787200.0&searchFor=all
http://erddap.emodnet-physics.eu/erddap/search/advanced.csv?page=1&itemsPerPage=1000&protocol=(ANY)&cdm_data_type=(ANY)&institution=(ANY)&ioos_category=(ANY)&keywords=(ANY)&long_name=(ANY)&standard_name=sea_water_salinity&variableName=(ANY)&minLon=-72.0&maxLon=-69&minLat=38&maxLat=41&minTime=1468108800.0&maxTime=1470787200.0

In [8]:
all_datasets.head

<bound method NDFrame.head of                                       server                      Dataset ID  \
38   https://coastwatch.pfeg.noaa.gov/erddap                  scrippsGliders   
40   https://coastwatch.pfeg.noaa.gov/erddap                    erdGtsppBest   
41   https://coastwatch.pfeg.noaa.gov/erddap                  erdGodaeSfcobs   
75   https://coastwatch.pfeg.noaa.gov/erddap                     cwwcNDBCMet   
93   https://coastwatch.pfeg.noaa.gov/erddap                    ndbcSosWTemp   
94   https://coastwatch.pfeg.noaa.gov/erddap                     ndbcSosWind   
95   https://coastwatch.pfeg.noaa.gov/erddap                 ndbcSosCurrents   
96   https://coastwatch.pfeg.noaa.gov/erddap                 ndbcSosSalinity   
97   https://coastwatch.pfeg.noaa.gov/erddap                   ndbcSosWLevel   
98   https://coastwatch.pfeg.noaa.gov/erddap                     nosSosATemp   
99   https://coastwatch.pfeg.noaa.gov/erddap                     nosSosBPres   
100  https

In [9]:
print('%i datasets found.' %all_datasets.shape[0])

60 datasets found.


In [10]:
df_coords = pd.DataFrame()
for i in range(all_datasets.shape[0]):
    server_url = all_datasets['server'].iloc[int(i)]
    dataset_id = all_datasets['Dataset ID'].iloc[int(i)]
    
    if "ROMS" in dataset_id or "DOP" in dataset_id: # skip ROMS model output
        print("Skipping %s" % server_url + dataset_id)
        continue
    #if dataset_id in df_coords['Dataset ID']:
    #    continue
    print(i)
    e2 = ERDDAP(
                 server=server_url,
                 protocol='tabledap',
                 response='csv'
           )
    e2.variables=["latitude","longitude"]#,"time"]
    e2.dataset_id = all_datasets['Dataset ID'].iloc[int(i)]
    #e2.constraints = kw
    e2.constraints = {
           "time>=": time_min,
           "time<=": time_max,
           "longitude>=": bbox[0],
           "longitude<=": bbox[1],
           "latitude>=": bbox[2],
           "latitude<=": bbox[3],
           "distinct" : ()
    }
    r = requests.get(e2.get_download_url())
    try:
        r.raise_for_status()
        print(e2.get_download_url())
        df = e2.to_pandas()
        print("Found %i unique coordinates." % df.shape[0])
        df['dataset_count'] = i
        df['dataset_download_url'] = e2.get_download_url()
        df['Dataset ID'] = dataset_id

        df_coords = pd.concat([df_coords,df])
    except requests.exceptions.HTTPError as err:
        print("HTTPError data not within bounds!!! {}".format(e2.get_download_url()))
    except requests.exceptions.RequestException as err:
        print("RequestException data not within bounds!!! {}".format(e2.get_download_url()))
    except requests.exceptions.ConnectionError as err:
        print("ConnectionError data not within bounds!!! {}".format(e2.get_download_url()))
    except requests.exceptions.Timeout as err:
        print("Timeout data not within bounds!!! {}".format(e2.get_download_url()))
    except requests.exceptions.ConnectTimeout as err:
        print("ConnectionTimeout data not within bounds!!! {}".format(e2.get_download_url()))
    

    #print(e.get_download_url(response="csv"))
    
    #dataset_url = '%s/tabledap/%s.csvp?latitude,longitude,time&longitude>=-72.0&longitude<=-69&latitude>=38&latitude<=41&time>=1278720000.0&time<=1470787200.0&distinct()' % (all_datasets['server'].iloc[int(i)],all_datasets['Dataset ID'].iloc[int(i)])

df_coords.head()
print("\n\nCollected %i unique coordinate pairs from %i datasets" % 
      (df_coords.shape[0], len(df_coords['dataset_count'].unique())))

0
HTTPError data not within bounds!!! https://coastwatch.pfeg.noaa.gov/erddap/tabledap/scrippsGliders.csv?latitude,longitude&time>=1468108800.0&time<=1470787200.0&longitude>=-72.0&longitude<=-69&latitude>=38&latitude<=41&distinct()
1
https://coastwatch.pfeg.noaa.gov/erddap/tabledap/erdGtsppBest.csv?latitude,longitude&time>=1468108800.0&time<=1470787200.0&longitude>=-72.0&longitude<=-69&latitude>=38&latitude<=41&distinct()
Found 6 unique coordinates.
2
https://coastwatch.pfeg.noaa.gov/erddap/tabledap/erdGodaeSfcobs.csv?latitude,longitude&time>=1468108800.0&time<=1470787200.0&longitude>=-72.0&longitude<=-69&latitude>=38&latitude<=41&distinct()
Found 247 unique coordinates.
3
https://coastwatch.pfeg.noaa.gov/erddap/tabledap/cwwcNDBCMet.csv?latitude,longitude&time>=1468108800.0&time<=1470787200.0&longitude>=-72.0&longitude<=-69&latitude>=38&latitude<=41&distinct()
Found 2 unique coordinates.
4
HTTPError data not within bounds!!! https://coastwatch.pfeg.noaa.gov/erddap/tabledap/ndbcSosWTemp

Found 1 unique coordinates.
36
http://www.myroms.org:8080/erddap/tabledap/PIONEER_WFP_TEMP_SALT_CP02PMCO.csv?latitude,longitude&time>=1468108800.0&time<=1470787200.0&longitude>=-72.0&longitude<=-69&latitude>=38&latitude<=41&distinct()
Found 1 unique coordinates.
37
http://www.myroms.org:8080/erddap/tabledap/PIONEER_WFP_TEMP_SALT_CP02PMUI.csv?latitude,longitude&time>=1468108800.0&time<=1470787200.0&longitude>=-72.0&longitude<=-69&latitude>=38&latitude<=41&distinct()
Found 1 unique coordinates.
38
http://www.myroms.org:8080/erddap/tabledap/PIONEER_WFP_TEMP_SALT_CP02PMUO.csv?latitude,longitude&time>=1468108800.0&time<=1470787200.0&longitude>=-72.0&longitude<=-69&latitude>=38&latitude<=41&distinct()
Found 1 unique coordinates.
39
http://www.myroms.org:8080/erddap/tabledap/PIONEER_WFP_TEMP_SALT_CP04OSPM.csv?latitude,longitude&time>=1468108800.0&time<=1470787200.0&longitude>=-72.0&longitude<=-69&latitude>=38&latitude<=41&distinct()
Found 1 unique coordinates.
40
HTTPError data not within bou

In [11]:
df_coords.head()

,latitude (degrees_north),longitude (degrees_east),dataset_count,dataset_download_url,Dataset ID
0,39.025,-71.711,1,https://coastwatch.pfeg.noaa.gov/erddap/tabled...,erdGtsppBest
1,39.244,-71.315,1,https://coastwatch.pfeg.noaa.gov/erddap/tabled...,erdGtsppBest
2,39.296,-70.464,1,https://coastwatch.pfeg.noaa.gov/erddap/tabled...,erdGtsppBest
3,39.645,-71.664,1,https://coastwatch.pfeg.noaa.gov/erddap/tabled...,erdGtsppBest
4,39.668,-70.735,1,https://coastwatch.pfeg.noaa.gov/erddap/tabled...,erdGtsppBest


In [12]:
#import matplotlib.pyplot as plt
#plt.figure(figsize=(12, 12)) #make_map(extent, figsize=(6, 6))\n",
#add_etopo2(extent, ax)\n",
#plt.scatter(df_coords['longitude (degrees_east)'],
#               df_coords['latitude (degrees_north)'],
#               s=3, c=df_coords['dataset_count'].values)
#plt.colorbar()
#plt.title("%i datasets" % len(df_coords['dataset_count'].unique()))
#df.plot(x='longitude (degrees_east)',y='latitude (degrees_north)')"

In [13]:
#df_coords.to_csv('coordinates.csv')
# need to skip ROMS data: http://tds.marine.rutgers.edu/erddap/tabledap/ROMSSobs.htmlTable?latitude,longitude,time&time%3E=1468108800.0&time%3C=1470787200.0&longitude%3E=-72.0&longitude%3C=-69&latitude%3E=38&latitude%3C=41&distinct()

In [14]:
#df_coords.to_csv('coordinates.zip',compression='zip')

In [15]:
# from erddapy import ERDDAP


# e = ERDDAP(
#   server='https://data.ioos.us/gliders/erddap',
#   protocol='tabledap',
# )

# e.response = 'csv'
# e.dataset_id = 'whoi_406-20160902T1700'
# e.constraints = {
#     'time>=': '2016-07-10T00:00:00Z',
#     'time<=': '2017-02-10T00:00:00Z',
#     'latitude>=': 38.0,
#     'latitude<=': 41.0,
#     'longitude>=': -72.0,
#     'longitude<=': -69.0,
# }
# e.variables = [
#     'depth',
#     'latitude',
#     'longitude',
#     'salinity',
#     'temperature',
#     'time',
# ]

In [16]:
#     if url != 'http://erddap.emodnet-physics.eu/erddap' and \
#     url != 'https://erddap.marine.ie/erddap' and \
#     url != 'http://oos.soest.hawaii.edu/erddap' and \
#     url != 'http://erddap.secoora.org/erddap' and \
#     url != 'https://ecowatch.ncddc.noaa.gov/erddap' and \
#     url != 'http://dap.onc.uvic.ca/erddap':

In [17]:
# laturl = 'http://erddap.bco-dmo.org/erddap/categorize/variableName/latitude/index.csv'
# lonurl = 'http://erddap.bco-dmo.org/erddap/categorize/variableName/longitude/index.csv'

# df_lat = pd.read_csv(laturl, header=[0])
# df_lon = pd.read_csv(lonurl, header=[0])

# df_datasets = pd.merge(df_lat, df_lon, on='Dataset ID')
# df_final = pd.DataFrame(columns=['latitude (degrees_north)', 'longitude (degrees_east)', 'Dataset ID'])

# for did in df_datasets['Dataset ID']:
#     if did != 'bcodmo_dataset_739309': # remove this once we reload all the datasets again.
#         dataset_url = 'http://erddap.bco-dmo.org/erddap/tabledap/%s.csvp?latitude,longitude&distinct()' % did
#         df_data = pd.read_csv(dataset_url, header=0, usecols=['latitude (degrees_north)', 'longitude (degrees_east)'])
#         df_data['Dataset ID'] = did
#         df_final = pd.concat([df_final, df_data], sort=False, ignore_index=True)
#         print(df_final.shape)

# lon = df_final['longitude (degrees_east)'].values
# lat = df_final['latitude (degrees_north)'].values